In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import glob
import pandas as pd
import numpy as np
from sglm.helpers import filehelpers as fh, dfhelpers as dfh
from pathlib import Path

In [4]:
base_location = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\outputs')

load_folder = base_location / Path(r'Figure_1_2')
h5_recons_locations = str((load_folder / Path(r'all\f1*-ft\reconstructs\best_resids_*.h5')).resolve())
# load_folder = base_location / Path(r'Figure_3')
# h5_beta_locations = str((load_folder / Path(r'all\f3*-ft\reconstructs\best_resids_*.h5')).resolve())
# load_folder = base_location / Path(r'Figure_4_g1')
# h5_beta_locations = str((load_folder / Path(r'all\f4*-ft\reconstructs\best_resids_*.h5')).resolve())
# load_folder = base_location / Path(r'Figure_4_g2')
# h5_beta_locations = str((load_folder / Path(r'all\f4*-ft\reconstructs\best_resids_*.h5')).resolve())
# load_folder = base_location / Path(r'Figure_5_g1')
# h5_beta_locations = str((load_folder / Path(r'all\f5*-ft\reconstructs\best_resids_*.h5')).resolve())
# load_folder = base_location / Path(r'Figure_5_g2')
# h5_beta_locations = str((load_folder / Path(r'all\f5*-ft\coefs\*_best_coeffs.h5')).resolve())

out_folder = base_location / Path(r'final_outputs')

out_loc = str((out_folder).resolve())
h5_recons_locations = glob.glob(h5_recons_locations)

In [5]:
base_keys = list({
                'gACH':(0,0),
                'rDA':(0,0),
                'gDA':(0,0),
#                 'photometryCenterInIndex':(0,0),
#                 'photometryCenterOutIndex':(0,0),
# #                 'photometrySideInIndex':(0,0),
                'photometrySideInIndexr':(0,0),
                'photometrySideInIndexnr':(0,0),
#                 'photometrySideOutIndex':(0,0),
#                 'sl': (0,0),
#                 'spnnrOff': (0,0),
#                 'photometryCenterInIndex':(0,0),
#                 'photometryCenterOutIndex':(0,0),
                'photometrySideInIndexAA':(0,0),
                'photometrySideInIndexAa':(0,0),
                'photometrySideInIndexaA':(0,0),
                'photometrySideInIndexaa':(0,0),
                'photometrySideInIndexAB':(0,0),
                'photometrySideInIndexAb':(0,0),
                'photometrySideInIndexaB':(0,0),
                'photometrySideInIndexab':(0,0),
#                 'photometrySideOutIndex':(0,0),
#                 'sl': (0,0),
#                 'spnnrOff': (0,0),
                }.keys())
shortened_keys = [dfh.shorten_col_name(_) for _ in base_keys]
# base_keys

In [ ]:



resid_df = resid_df[~resid_df['dupe']]

# display(resid_df)
# break

b0 = np.arange(len(resid_df))
# display(b0)

rdc = []

lb = -40
ub = 100

pred_params_cols = [_ for _ in resid_df.columns if 'pred_paramsNum=' in _]
print(pred_params_cols)

if len([_ for _ in resid_df.columns if 'run_num=9' in _]) < 1:
    continue

true_col = pred_params_cols[0].replace('pred_paramsNum=0_', '').split('_')[0]
print('true_col', true_col)

for event_col in ['photometrySideInIndexr']:
    resid_df_cutdown = resid_df[['Unnamed: 0', 'file_num', 'dupe', 'nTrial', 'nEndTrial', 'wi_trial_keep',
                                'gDA', 'gACH', 'rDA', event_col] + pred_params_cols].copy()

    event = np.where(resid_df_cutdown[event_col].values == 1)[0]
    print(b0.shape, event.shape)
    for iev, ev in enumerate(event):
        delta = b0 - ev
        # print(delta)
        keep_eg = (delta >= -40)&(delta <= 100)&(resid_df['is_holdout'])
        delta_keep_eg = delta[keep_eg]
        rdf_tmp = resid_df_cutdown.loc[keep_eg].copy()
        rdf_tmp['delta'] = delta_keep_eg
        # rdf_tmp = rdf_tmp[~rdf_tmp['dupe']]
        # display(rdf_tmp)
        # display(list(rdf_tmp.columns))
        rdc.append(rdf_tmp.set_index('delta'))

    # rdc_df = pd.concat(rdc, axis=0)
    # display(rdc_df)
    # rdc_df = rdc_df[[true_col]]
    # delta_index.append(delta)
    # delta_val.append((bs.confidence_interval.low, mn_val, bs.confidence_interval.high))

    reconstruction_data_list = []

    seed = np.random.randint(1000000000)
    for bootstrap_col in [true_col] + pred_params_cols:
        rdc_lst = []
        for rdc_df in rdc:
            rdc_lst.append(rdc_df[[bootstrap_col]])
        rdc_df = pd.concat(rdc_lst, axis=1)

        for delta in rdc_df.index:
            row = rdc_df.loc[delta].dropna().values
            mn_val = row.mean()
            bs = scipy.stats.bootstrap((row,), np.mean, n_resamples=9999,
                                       batch=None, vectorized=True, paired=False,
                                       axis=0, confidence_level=0.95, method='BCa',
                                       random_state=seed)

            rtup = (bootstrap_col, delta, bs.confidence_interval.low, mn_val, bs.confidence_interval.high)
            reconstruction_data_list.append(rtup)
            print(reconstruction_data_list[-1])

In [6]:
h5_lst = []
for h5_coef_fn in h5_recons_locations:
    h5f = pd.HDFStore(h5_coef_fn)
    h5fk = h5f.keys()
    for key in h5fk:
        if 'run_num' in key and 'run_num__0' not in key:
            continue
        print(key)
        
        y_col = key.split('_')[0].replace('/', '')
        h5_df = pd.read_hdf(h5f, key)
        
        h5_df.columns = [dfh.shorten_col_name(_) for _ in h5_df.columns]
        
        if 'dfrel_basis' != key.replace('/', ''):
            h5_df.columns = [_.split('_')[0] for _ in h5_df.columns]
            h5_df['true'] = h5_lst[0][y_col]
            h5_df['resid2'] = (h5_df['true'] - h5_df['pred'])**2
            
            h5_df.columns = [_.split('_')[0] for _ in h5_df.columns]
            h5_lst.append(h5_df)
            break
        else:
            h5_lst.append(h5_df)
        
    break
# concat = pd.concat(h5_lst, axis=0)

/dfrel_basis
/gACH_0_0_base_simple_run_num__0


In [7]:
h5_lst[0][shortened_keys]

,,gACH,rDA,gDA,SIr,SInr,SIAA,SIAa,SIaA,SIaa,SIAB,SIAb,SIaB,SIab
,nTrial_filenum,,,,,,,,,,,,,
0,1,1.291100,-1.830134,-1.403055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.948091,-1.206838,-1.186573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1.808744,-1.191822,-0.707389,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1.693770,-0.478138,0.190341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,-0.079142,-0.593920,-0.526820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2168706,63296,-0.552477,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2168707,63296,0.858026,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2168708,63296,1.403134,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
h5_lst[1]

,,holdout,pred,predALL,true,resid2
,nTrial_filenum,,,,,
0,1,0.0,0.226690,0.226690,1.291100,1.132968
1,1,0.0,0.234288,0.234288,1.948091,2.937120
2,1,0.0,0.109966,0.109966,1.808744,2.885846
3,1,0.0,0.035002,0.035002,1.693770,2.751511
4,1,0.0,0.015932,0.015932,-0.079142,0.009039
...,...,...,...,...,...,...
2168706,63296,1.0,-0.235383,-0.235383,-0.552477,0.100548
2168707,63296,1.0,-0.023928,-0.023928,0.858026,0.777844
2168708,63296,1.0,0.266767,0.266767,1.403134,1.291329
